<a href="https://colab.research.google.com/github/AdityaPareek22/Indian-Number-Plate-Recognition-System/blob/master/model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import cv2
import os
import sys
import glob
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage import io
import skimage
import sklearn
import pickle
from tqdm import tqdm
import gc
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential,  Model
from keras.layers import Dense,  Dropout,  Flatten,  Input
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, ZeroPadding2D
from keras.constraints import maxnorm
from keras.optimizers import SGD,  Adam
from keras.layers.convolutional import Conv2D,  MaxPooling2D, Convolution2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.preprocessing.image import ImageDataGenerator

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [0]:
os.chdir(os.path.join('.','gdrive','My Drive','grid'))

In [0]:
data = pd.read_csv('./training.csv')

In [0]:
train_dir = './train_data'

In [0]:
datagen = ImageDataGenerator(validation_split=0.2)

train_generator = datagen.flow_from_dataframe(dataframe=data,
                                              directory=train_dir,
                                              subset='training',
                                              x_col=data.columns[0], 
                                              y_col=data.columns[1:], 
                                              target_size=(480,640),
                                              color_mode='rgb',
                                              class_mode='other',
                                              batch_size=12,
                                              shuffle=True,
                                              seed=22)

validation_generator = datagen.flow_from_dataframe(dataframe=data,
                                              directory=train_dir,
                                              subset='validation',
                                              x_col=data.columns[0], 
                                              y_col=data.columns[1:], 
                                              target_size=(480,640),
                                              color_mode='rgb',
                                              class_mode='other',
                                              batch_size=12,
                                              shuffle=True,
                                              seed=22)

Found 11200 images.
Found 2800 images.


In [0]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=(480,640,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(GlobalAveragePooling2D())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.compile(optimizer=Adam(lr=1e-3,decay=1e-6),loss='mse',metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

In [0]:
# Callbacks
early_stopping_monitor = EarlyStopping(patience=3)
filepath='model_2.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint, early_stopping_monitor]

In [0]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    use_multiprocessing=True,
                    callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
933/933 [==============================] - 4067s 4s/step - loss: 8511.4824 - acc: 0.8990 - val_loss: 5494.8879 - val_acc: 0.8991

Epoch 00001: val_acc improved from -inf to 0.89914, saving model to model_2.h5
Epoch 2/10
933/933 [==============================] - 525s 563ms/step - loss: 6912.9821 - acc: 0.9045 - val_loss: 5676.5520 - val_acc: 0.9003

Epoch 00002: val_acc improved from 0.89914 to 0.90029, saving model to model_2.h5
Epoch 3/10
933/933 [==============================] - 519s 556ms/step - loss: 6505.0711 - acc: 0.9043 - val_loss: 5384.0141 - val_acc: 0.8981

Epoch 00003: val_acc did not improve from 0.90029
Epoch 4/10
933/933 [==============================] - 524s 562ms/step - loss: 6178.2990 - acc: 0.9043 - val_loss: 4910.5042 - val_acc: 0.8996

Epoch 00004: val_acc did not improve from 0.90029
Epoch 5/10
933/933 [==============================] - 525s 563ms/step - loss: 5616.8648 - acc: 0.9040 - val_loss: 6054.88